In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

# Dados de exemplo
np.random.seed(42)
data = []

clubes = [f'Clube_{i}' for i in range(1, 11)]
posicoes = ['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral']

for clube in clubes:
    for posicao in posicoes:
        for _ in range(3):
            data.append({
                'Clube': clube,
                'Posicao': posicao,
                'Cartoes': np.random.randint(1, 15),
                'Minuto': np.random.randint(0, 91)
            })

df = pd.DataFrame(data)

# Treemap com todas as 4 variáveis
fig = px.treemap(df,
                 path=['Clube', 'Posicao'],  # Hierarquia
                 values='Cartoes',  # Tamanho dos retângulos
                 color='Minuto',  # Cor = minuto da falta
                 color_continuous_scale='Reds',
                 title='Treemap: Clube → Posição (tamanho=Cartões, cor=Minuto)',
                 hover_data=['Cartoes', 'Minuto'])

fig.update_traces(textinfo='label+value')
fig.update_layout(height=700, width=1000)
fig.show()


In [2]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Preparar dados agregados
np.random.seed(42)
clubes = [f'C{i}' for i in range(1, 9)]
posicoes = ['Gol', 'Zag', 'Meio', 'Ata', 'Lat']

# Criar matriz de conexões
n = len(clubes)
m = len(posicoes)
matrix = np.random.randint(1, 20, size=(n, m))

# Converter para formato de links
source = []
target = []
value = []
minuto_medio = []

for i, clube in enumerate(clubes):
    for j, pos in enumerate(posicoes):
        if matrix[i, j] > 0:
            source.append(clube)
            target.append(pos)
            value.append(matrix[i, j])
            minuto_medio.append(np.random.randint(0, 91))

df_links = pd.DataFrame({
    'source': source,
    'target': target,
    'value': value,
    'minuto': minuto_medio
})

# Criar lista de nós únicos
all_nodes = list(set(source + target))
node_dict = {node: idx for idx, node in enumerate(all_nodes)}

# Converter para índices
df_links['source_idx'] = df_links['source'].map(node_dict)
df_links['target_idx'] = df_links['target'].map(node_dict)

# Criar Sankey (melhor que chord para este caso)
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color='black', width=0.5),
        label=all_nodes,
        color='lightblue'
    ),
    link=dict(
        source=df_links['source_idx'],
        target=df_links['target_idx'],
        value=df_links['value'],
        color=['rgba(255,' + str(int(255*(1-m/90))) + ',0,0.4)' 
               for m in df_links['minuto']],
        customdata=df_links['minuto'],
        hovertemplate='%{source.label} → %{target.label}<br>'+
                      'Cartões: %{value}<br>Minuto médio: %{customdata}<extra></extra>'
    )
)])

fig.update_layout(
    title='Sankey Diagram: Fluxo Clube → Posição (espessura=Cartões, cor=Minuto)',
    height=700,
    width=1000
)

fig.show()


In [3]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Dados agregados
np.random.seed(42)
clubes = [f'Clube_{i}' for i in range(1, 13)]
posicoes = ['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral']

# Criar dados para cada combinação
data = []
for clube in clubes:
    for posicao in posicoes:
        cartoes = np.random.randint(0, 25)
        minuto = np.random.randint(0, 91)
        data.append({
            'Clube': clube,
            'Posicao': posicao,
            'Cartoes': cartoes,
            'Minuto': minuto
        })

df = pd.DataFrame(data)

# Criar matriz pivot
pivot_cartoes = df.pivot(index='Clube', columns='Posicao', values='Cartoes')
pivot_minutos = df.pivot(index='Clube', columns='Posicao', values='Minuto')

# Criar texto personalizado para hover
hover_text = []
for i in range(len(pivot_cartoes.index)):
    hover_row = []
    for j in range(len(pivot_cartoes.columns)):
        hover_row.append(
            f'Clube: {pivot_cartoes.index[i]}<br>' +
            f'Posição: {pivot_cartoes.columns[j]}<br>' +
            f'Cartões: {pivot_cartoes.iloc[i, j]}<br>' +
            f'Minuto médio: {pivot_minutos.iloc[i, j]}'
        )
    hover_text.append(hover_row)

# Criar heatmap com anotações
fig = go.Figure(data=go.Heatmap(
    z=pivot_cartoes.values,
    x=pivot_cartoes.columns,
    y=pivot_cartoes.index,
    colorscale='YlOrRd',
    text=pivot_minutos.values,
    texttemplate='%{text}min',
    hovertext=hover_text,
    hoverinfo='text',
    colorbar=dict(title='Quantidade<br>de Cartões')
))

fig.update_layout(
    title='Heatmap: Cartões por Clube e Posição (números=Minuto médio)',
    xaxis_title='Posição do Jogador',
    yaxis_title='Clube',
    height=700,
    width=900
)

fig.show()


In [4]:
import plotly.express as px
import pandas as pd
import numpy as np

# Dados de exemplo
np.random.seed(42)
n_samples = 200

df = pd.DataFrame({
    'Minuto': np.random.randint(0, 91, n_samples),
    'Cartoes': np.random.randint(1, 15, n_samples),
    'Clube': np.random.choice([f'Clube_{i}' for i in range(1, 7)], n_samples),
    'Posicao': np.random.choice(['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral'], n_samples)
})

fig = px.scatter(df,
                 x='Minuto',
                 y='Cartoes',
                 size='Cartoes',
                 color='Posicao',
                 facet_col='Clube',
                 facet_col_wrap=3,
                 title='Bubble Chart com Facetas: Minuto × Cartões por Clube e Posição',
                 labels={'Minuto': 'Minuto do Jogo', 'Cartoes': 'Quantidade de Cartões'},
                 height=800,
                 category_orders={'Clube': sorted(df['Clube'].unique())})

fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))
fig.show()


In [5]:
import plotly.express as px
import pandas as pd
import numpy as np

# Dados hierárquicos
np.random.seed(42)
data = []

clubes = [f'Clube_{i}' for i in range(1, 11)]
posicoes = ['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral']
periodos = ['0-30min', '31-60min', '61-90min']

for clube in clubes:
    for posicao in posicoes:
        for periodo in periodos:
            data.append({
                'Clube': clube,
                'Posicao': posicao,
                'Periodo': periodo,
                'Cartoes': np.random.randint(1, 10),
                'Minuto_medio': np.random.randint(0, 91)
            })

df = pd.DataFrame(data)

# Icicle chart (sunburst retangular)
fig = px.icicle(df,
                path=['Clube', 'Posicao', 'Periodo'],
                values='Cartoes',
                color='Minuto_medio',
                color_continuous_scale='Reds',
                title='Icicle Chart: Hierarquia Clube → Posição → Período (cor=Minuto)',
                hover_data=['Cartoes', 'Minuto_medio'])

fig.update_traces(textinfo='label+value')
fig.update_layout(height=700, width=1200)
fig.show()


In [6]:
import plotly.express as px
import pandas as pd
import numpy as np

# Dados categorizados
np.random.seed(42)
n_samples = 300

df = pd.DataFrame({
    'Clube': np.random.choice([f'Clube_{i}' for i in range(1, 8)], n_samples),
    'Posicao': np.random.choice(['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral'], n_samples),
    'Periodo': np.random.choice(['0-30min', '31-60min', '61-90min'], n_samples),
    'Cartoes_Faixa': np.random.choice(['1-5', '6-10', '11-15'], n_samples),
    'Cartoes': np.random.randint(1, 16, n_samples)
})

# Agregar dados
df_agg = df.groupby(['Clube', 'Posicao', 'Periodo', 'Cartoes_Faixa']).size().reset_index(name='Count')

fig = px.parallel_categories(df,
                             dimensions=['Clube', 'Posicao', 'Periodo', 'Cartoes_Faixa'],
                             color='Cartoes',
                             color_continuous_scale='YlOrRd',
                             title='Parallel Categories: Fluxo entre Todas as Variáveis',
                             labels={'Clube': 'Clube', 
                                    'Posicao': 'Posição',
                                    'Periodo': 'Minuto',
                                    'Cartoes_Faixa': 'Cartões'})

fig.update_layout(height=600, width=1200)
fig.show()


In [7]:
import plotly.express as px
import pandas as pd
import numpy as np

# Dados agregados
np.random.seed(42)
data = []

clubes = [f'Clube_{i}' for i in range(1, 11)]
posicoes = ['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral']
periodos = ['1º Tempo', '2º Tempo']

for clube in clubes:
    for posicao in posicoes:
        for periodo in periodos:
            data.append({
                'Clube': clube,
                'Posicao': posicao,
                'Periodo': periodo,
                'Cartoes': np.random.randint(0, 12)
            })

df = pd.DataFrame(data)

fig = px.bar(df,
             x='Clube',
             y='Cartoes',
             color='Posicao',
             facet_row='Periodo',
             title='Stacked Bar: Cartões por Clube, Posição e Período',
             labels={'Cartoes': 'Quantidade de Cartões'},
             height=700,
             barmode='stack')

fig.update_xaxes(tickangle=45)
fig.show()


In [9]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.spatial import distance

# Dados hierárquicos para circle packing
np.random.seed(42)

ids = ['Total']
labels = ['Total']
parents = ['']
values = [0]
minutos = [45]

clubes = [f'Clube_{i}' for i in range(1, 9)]
posicoes = ['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral']

# Adicionar clubes
for clube in clubes:
    ids.append(clube)
    labels.append(clube)
    parents.append('Total')
    values.append(0)
    minutos.append(np.random.randint(30, 60))

# Adicionar posições dentro de clubes
for clube in clubes:
    for posicao in posicoes:
        id_pos = f'{clube}_{posicao}'
        ids.append(id_pos)
        labels.append(posicao)
        parents.append(clube)
        values.append(np.random.randint(3, 12))
        minutos.append(np.random.randint(0, 91))

df_pack = pd.DataFrame({
    'ids': ids,
    'labels': labels,
    'parents': parents,
    'values': values,
    'minutos': minutos
})

# Usar treemap como alternativa (circle packing não está diretamente em plotly)
fig = px.treemap(df_pack,
                 ids='ids',
                 names='labels',
                 parents='parents',
                 values='values',
                 color='minutos',
                 color_continuous_scale='Reds',
                 title='Treemap Hierárquico: Clube → Posição (cor=Minuto médio)',
                 hover_data=['values', 'minutos'])

fig.update_traces(textinfo='label+value', marker=dict(cornerradius=5))
fig.update_layout(height=700, width=1000)
fig.show()


In [10]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Dados para distribuições
np.random.seed(42)
posicoes = ['Goleiro', 'Zagueiro', 'Meio-campo', 'Atacante', 'Lateral']

fig = go.Figure()

for i, posicao in enumerate(posicoes):
    minutos = np.random.normal(45, 20, 100)
    minutos = np.clip(minutos, 0, 90)
    
    fig.add_trace(go.Violin(
        x=minutos,
        y=[posicao] * len(minutos),
        name=posicao,
        orientation='h',
        side='positive',
        width=3,
        points=False,
        line_color='red',
        fillcolor='rgba(255,0,0,0.3)'
    ))

fig.update_layout(
    title='Ridgeline: Distribuição de Minutos das Faltas por Posição',
    xaxis_title='Minuto do Jogo',
    yaxis_title='Posição',
    height=600,
    width=1000,
    showlegend=False
)

fig.show()
